In [ ]:
from __future__ import print_function
import pickle
from os import path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
folder_id = '16HNqeTaB6EVkupDW_ENv3xLs2A7Sk3Ml'

creds = None
if path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow \
                .from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('drive', 'v3', credentials=creds)
items = service \
        .files() \
        .list(
            q=f'"{folder_id}" in parents',
            fields='incompleteSearch, files(id, name, parents)') \
        .execute() \
        .get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(', '.join(map(str, item.values())))